<a href="https://colab.research.google.com/github/jaredschlak/CleaningData/blob/main/DataCleaningProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#Important libraries imported below

import numpy as np # linear algebra
import pandas as pd # data processing
import os

In [11]:
#Create a dataframe "df" that gets the house pricing data (Sourced from Kaggle.com)
df = pd.read_csv('/house_prices_records.csv')

#Shows a sampling of the data, missing values, and also describes the data like displaying the median, mode, min, max, etc.
pd.set_option('display.max_columns', None)
print(df.head(100))
df.info()
df.describe()

    1stFlrSF  2ndFlrSF  BedroomAbvGr BsmtExposure  BsmtFinSF1 BsmtFinType1  \
0        856     854.0           3.0           No         706          GLQ   
1       1262       0.0           3.0           Gd         978          ALQ   
2        920     866.0           3.0           Mn         486          GLQ   
3        961       NaN           NaN           No         216          ALQ   
4       1145       NaN           4.0           Av         655          GLQ   
..       ...       ...           ...          ...         ...          ...   
95       680     790.0           3.0           No         310          NaN   
96      1588       NaN           NaN           Av        1162          ALQ   
97       960       0.0           3.0           No         520          Rec   
98       835       0.0           2.0           No         108          ALQ   
99      1225       0.0           3.0           No         569          ALQ   

    BsmtUnfSF  EnclosedPorch  GarageArea GarageFinish  GarageYr

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtFinSF1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageYrBlt,GrLivArea,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
count,1460.000000,1374.000000,1361.000000,1460.000000,1460.000000,136.000000,1460.000000,1379.000000,1460.000000,1460.000000,1201.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,155.000000,1460.000000,1460.000000,1460.000000
mean,1162.626712,348.524017,2.869214,443.639726,567.240411,25.330882,472.980137,1978.506164,1515.463699,10516.828082,70.049958,103.685262,46.660274,5.575342,6.099315,1057.429452,103.741935,1971.267808,1984.865753,180921.195890
std,386.587738,438.865586,0.820115,456.098091,441.866955,66.684115,213.804841,24.689725,525.480383,9981.264932,24.284752,181.066207,66.256028,1.112799,1.382997,438.705324,135.543152,30.202904,20.645407,79442.502883
min,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1900.000000,334.000000,1300.000000,21.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1872.000000,1950.000000,34900.000000
25%,882.000000,0.000000,2.000000,0.000000,223.000000,0.000000,334.500000,1961.000000,1129.500000,7553.500000,59.000000,0.000000,0.000000,5.000000,5.000000,795.750000,0.000000,1954.000000,1967.000000,129975.000000
50%,1087.000000,0.000000,3.000000,383.500000,477.500000,0.000000,480.000000,1980.000000,1464.000000,9478.500000,69.000000,0.000000,25.000000,5.000000,6.000000,991.500000,0.000000,1973.000000,1994.000000,163000.000000
75%,1391.250000,728.000000,3.000000,712.250000,808.000000,0.000000,576.000000,2002.000000,1776.750000,11601.500000,80.000000,166.000000,68.000000,6.000000,7.000000,1298.250000,182.500000,2000.000000,2004.000000,214000.000000
max,4692.000000,2065.000000,8.000000,5644.000000,2336.000000,286.000000,1418.000000,2010.000000,5642.000000,215245.000000,313.000000,1600.000000,547.000000,9.000000,10.000000,6110.000000,736.000000,2010.000000,2010.000000,755000.000000


In [12]:
#In this data set there are 4 columns that are categorical data and not numerical.
#I wanted to make sure that the data was representative of what the data set said
#in it's metadata.  There were no 'outliers' in this categorical data or
#mispellings/typos.

unique_items = df['BsmtExposure'].unique()
print(unique_items)

unique_items = df['BsmtFinType1'].unique()
print(unique_items)

unique_items = df['GarageFinish'].unique()
print(unique_items)

unique_items = df['KitchenQual'].unique()
print(unique_items)

['No' 'Gd' 'Mn' 'Av' nan]
['GLQ' 'ALQ' 'Unf' 'Rec' nan 'BLQ' 'LwQ']
['RFn' 'Unf' nan 'Fin']
['Gd' 'TA' 'Ex' 'Fa']


In [20]:
#There are 10 columns that have missing values that we have to figure out what we should do.

# Identify the columns that have missing numbers where NaN can safely be replaced with 0.
# For example, If 2nd Flr Squar Footage is NaN, then it is fairly safe to assume that it doesn't have a 2nd Floor and we can replace that value with 0
#We can fix 5 columns listed below by filling in the NaN with 0.
cols_with_missing_numbers = ['2ndFlrSF', 'BedroomAbvGr', 'EnclosedPorch', 'MasVnrArea', 'WoodDeckSF']
df[cols_with_missing_numbers] = df[cols_with_missing_numbers].fillna(0)

#There are several missing values for LotFrontage.  To get a somewhat reasonable value for this, I know that LotArea is not missing any values.
#I am going to make an assumption that the LotArea is directly proportional to the LotFrontage.  Below I find the average ratio for those two
#columns and then create new values dy multiplying the LotArea by the ratio to get the new values.
#This is not perfect, but I believe it would be better than just leaving them blank.
#To find all rows from the two columns "LotArea" and "LotFrontage" where they have valid numbers we do the following
valid_rows = df[['LotFrontage', 'LotArea']].dropna()
#To find the average ratio between the two columns we can do the following
ratio = (valid_rows['LotFrontage'] / valid_rows['LotArea']).median()
# Fill the missing values NaN in "LotFrontage" by multiplying by the ratio with LotArea
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotArea'] * ratio)

#Next we can safely assume that if the Basement Finished SqFt and Basement Unfinished SqFt are both equal to 0,
#then we can put the Basmement Exposure equal to "No Basement"
condition = (df['BsmtFinSF1'] == 0) & (df['BsmtUnfSF'] == 0)
df.loc[condition, 'BsmtExposure'] = 'None'
#For the values where the Basement unfinished SqFt is non-zero, we find the mode of the Basement Exposure and fill in the NaN values
#of that column with the mode.  This is not the best fit, but it is better than leaving with NaN.
mode_value = df[df['BsmtUnfSF'] > 0]['BsmtExposure'].mode()
df['BsmtExposure'] = df['BsmtExposure'].fillna('No')

#For the NaN values of BsmtFinType1, we can safely assume that if the Total Basement SqFt is equal to 0, then the Finish Type is None.
condition = (df['BsmtFinType1'].isnull()) & (df['TotalBsmtSF'] == 0)
df.loc[condition, 'BsmtFinType1'] = 'None'

#For the NaN values that are left over, we filled in those values with the mode.
mode_value = df['BsmtFinType1'].mode().iloc[0]
df['BsmtFinType1'] = df['BsmtFinType1'].fillna(mode_value)

#For the column GarageFinish, if the GarageArea is equal to 0, we can assume that the GarageFinish is None, since it has no SqFt.
condition = (df['GarageFinish'].isnull()) & (df['GarageArea'] == 0)
df.loc[condition, 'GarageFinish'] = 'None'

# For the column GarageFinish, I did some conditional fills.  If the OverallQual of the house was 8 or better then I filled it with Finished
#If the OverallQual was between 5 and 7 I put the Garage Finish as RoughlyFinished
#If the OverallQual was less than 5, I put the GarageFinish as Unfinished.
#This is of course an assumption, but I feel by the definition of the column OverallQual: Rates the overall material and finish of the house
#It would be somewhat safe to say that if the finish of the house is a 10, then the garage has a better chance of also being finished.
missing_gar_finish = df['GarageFinish'].isnull()
# Apply conditional fills to the GarageFinish Column
df.loc[missing_gar_finish & (df['OverallQual'] >= 8), 'GarageFinish'] = 'Fin'
df.loc[missing_gar_finish & (df['OverallQual'] < 8) & (df['OverallQual'] >= 5), 'GarageFinish'] = 'RFn'
df.loc[missing_gar_finish & (df['OverallQual'] < 5), 'GarageFinish'] = 'Unf'

#I decided that for the missing values of GarageYrBlt, the best replacement would be to plug in the value where the house was actually built.
#Of course this might not be the case, but for most houses, the year the garage was built would be the same as the year the house was built.
#Also, I decided that for houses that did'nt have a garage, it would still be better to put down the year the house was built, instead of
#putting a 0 or a -1, as this might really skew the data.  There are other columns that take into consideration that the house does not have a garage.
df.loc[df['GarageYrBlt'].isnull(), 'GarageYrBlt'] = df['YearBuilt']

#I wanted to go ahead and show that all values are non-null after imputing some of the values.
#Also, I wanted to show the mean, median, mode, etc for the cleaned data.
df.info()
df.describe()

nan_rows = df[df['BsmtExposure'].isnull()]
print(nan_rows)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   1stFlrSF       1460 non-null   int64  
 1   2ndFlrSF       1460 non-null   float64
 2   BedroomAbvGr   1460 non-null   float64
 3   BsmtExposure   1460 non-null   object 
 4   BsmtFinSF1     1460 non-null   int64  
 5   BsmtFinType1   1460 non-null   object 
 6   BsmtUnfSF      1460 non-null   int64  
 7   EnclosedPorch  1460 non-null   float64
 8   GarageArea     1460 non-null   int64  
 9   GarageFinish   1460 non-null   object 
 10  GarageYrBlt    1460 non-null   float64
 11  GrLivArea      1460 non-null   int64  
 12  KitchenQual    1460 non-null   object 
 13  LotArea        1460 non-null   int64  
 14  LotFrontage    1460 non-null   float64
 15  MasVnrArea     1460 non-null   float64
 16  OpenPorchSF    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  OverallQ